<a href="https://colab.research.google.com/github/monsund/unsupervised-classification-of-Linkedin-Profiles-using-KMeans-LDA-TFIDF/blob/master/gensim_LDA_Mallet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install LDA Mallet Package**

In [ ]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip

In [ ]:
mallet_path = '/content/mallet-2.0.8/bin/mallet'

# **Load LDA Corpus**

In [ ]:
import pickle
with open("/content/drive/My Drive/NLP/with_swaraj/Data/lda_corpus_bigram_threshold_70.txt", "rb") as fp:   # Unpickling
  corpus = pickle.load(fp)

In [ ]:
with open("/content/drive/My Drive/NLP/with_swaraj/Data/lda_id2word_bigram_threshold_70.txt", "rb") as fp:   # Unpickling
  id2word = pickle.load(fp)

In [ ]:
with open("/content/drive/My Drive/NLP/with_swaraj/Data/data_list_for_lda_dictionary.json", "rb") as fp:   # Unpickling
  data_list_for_lda_dictionary = pickle.load(fp)

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/NLP/with_swaraj/Data/linkedin_about_no_foreign_lang.csv')
df = df.dropna()
df = df.drop(['cleaned_about_us', 'cluster'], axis=1)
df = df.reset_index(drop=True)
df.head()

,url,username,title,about
0,https://www.linkedin.com/in/lokesh-kumar-xess-...,lokesh-kumar-xess-54814068,Marketing & Strategy at Mobistreak,I am a Google Adwords and Google Analytics cer...
1,https://in.linkedin.com/in/sujithnarayanan,sujithnarayanan,Reimagining Financial Services for India,Ideate. Execute. Disrupt. Iterate.
2,https://www.linkedin.com/in/adarshramakrishnan,adarshramakrishnan,Product Strategist | Product Manager | Triple ...,"Over the past decade, I have co-founded 3 vent..."
3,https://www.linkedin.com/in/rhythm-bhatnagar-4...,rhythm-bhatnagar-4350b551,Product Marketer | Ex-SHEROES | Early-Stage St...,"Hey,I am Rhythm. I am passionate about startup..."
4,https://www.linkedin.com/in/karthiksureshlbs,karthiksureshlbs,Product at Facebook | CMU MS | LBS MBA,I am a product-centric builder with experience...


# **Build LDA Mallet Model**

In [ ]:
import gensim

In [ ]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=18, id2word=id2word)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
# display topics
ldamallet.print_topics()

[(0,
  '0.052*"work" + 0.041*"learn" + 0.041*"time" + 0.039*"good" + 0.033*"problem" + 0.028*"passionate" + 0.026*"challenge" + 0.025*"solve" + 0.023*"understand" + 0.020*"opportunity"'),
 (1,
  '0.027*"school" + 0.025*"social" + 0.024*"education" + 0.021*"world" + 0.018*"health" + 0.016*"student" + 0.016*"award" + 0.016*"university" + 0.015*"impact" + 0.015*"healthcare"'),
 (2,
  '0.050*"people" + 0.042*"make" + 0.037*"love" + 0.028*"life" + 0.023*"world" + 0.018*"thing" + 0.016*"find" + 0.016*"live" + 0.014*"great" + 0.014*"learn"'),
 (3,
  '0.191*"management" + 0.068*"customer" + 0.048*"manage" + 0.039*"process" + 0.034*"operation" + 0.031*"program" + 0.027*"client" + 0.025*"team" + 0.024*"support" + 0.020*"delivery"'),
 (4,
  '0.094*"team" + 0.047*"build" + 0.041*"lead" + 0.030*"leadership" + 0.028*"organization" + 0.028*"leader" + 0.027*"role" + 0.027*"people" + 0.025*"talent" + 0.024*"hr"'),
 (5,
  '0.169*"marketing" + 0.090*"digital" + 0.055*"brand" + 0.037*"medium" + 0.032*"con

# **calculate the coherence score**

In [ ]:
from gensim.models.coherencemodel import CoherenceModel

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_list_for_lda_dictionary, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('Coherence Score: ', coherence_ldamallet)

Coherence Score:  0.5177203109259058


# **Coherence Score for N number of topics**

In [ ]:
num_topics = [x for x in range(12,21,2)]
num_topics

[12, 14, 16, 18, 20]

In [ ]:
from gensim.models.coherencemodel import CoherenceModel

def get_coherence_score(model=ldamallet, num_topics_list=num_topics):
  coherence_score = []
  for num in num_topics:
    ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num, id2word=id2word)
    
    coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_list_for_lda_dictionary, dictionary=id2word, coherence='c_v')
    coherence_ldamallet = coherence_model_ldamallet.get_coherence()
    print(f'Coherence Score for {num}  topics : {coherence_ldamallet}')

    dic= {'no. of topics':num, 'coherence_score':coherence_ldamallet}
    coherence_score.append(dic)
  
  return coherence_score


In [ ]:
get_coherence_score(model=ldamallet, num_topics_list=num_topics)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Coherence Score for 12  topics : 0.5101425051979438
Coherence Score for 14  topics : 0.5113855810214268
Coherence Score for 16  topics : 0.514498337646117
Coherence Score for 18  topics : 0.5154999664261368
Coherence Score for 20  topics : 0.5141675518935245


[{'coherence_score': 0.5101425051979438, 'no. of topics': 12},
 {'coherence_score': 0.5113855810214268, 'no. of topics': 14},
 {'coherence_score': 0.514498337646117, 'no. of topics': 16},
 {'coherence_score': 0.5154999664261368, 'no. of topics': 18},
 {'coherence_score': 0.5141675518935245, 'no. of topics': 20}]

# **Topic Distribution**

In [ ]:
tm_results = ldamallet[corpus]

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
len(tm_results)

9044

# **Most probable words for the given Topic ID**

In [ ]:
# top probable words for topic id 1
print(f'top probable words of Topic 1: {ldamallet.show_topic(1, topn=10)}')

top probable words of Topic 1: [('school', 0.026901331387926317), ('social', 0.02548787160313697), ('education', 0.023800838956775486), ('world', 0.02060915557176728), ('health', 0.018055808863760715), ('student', 0.016004012401969723), ('award', 0.015912821448112346), ('university', 0.015684844063468904), ('impact', 0.01513769834032464), ('healthcare', 0.014590552617180376)]


In [ ]:
# Topic wise Top probable words with word probability
topics_with_word_prob = [[(term, round(wt, 3)) for term, wt in ldamallet.show_topic(n, topn=20)] for n in range(0, ldamallet.num_topics)]

In [ ]:
# Topic wise keywords only
topics_keywords = topics_with_word_prob = [[term for term, wt in ldamallet.show_topic(n, topn=20)] for n in range(0, ldamallet.num_topics)]

# **Find Dominant topic for a given documents**

In [ ]:
dominant_topics = [sorted(topics, key=lambda record: -record[1])[0] for topics in tm_results]

In [ ]:
dominant_topics_df = pd.DataFrame(dominant_topics, columns=['topic', 'topic_probability'])

In [ ]:
dominant_topics_df['topic'].isnull().values.any()
# dominant_topics_df.topic.tolist()

False

In [ ]:
df['dominant_topic'] = dominant_topics_df.topic.tolist()
df['topic_prob'] = dominant_topics_df.topic_probability.tolist()

In [ ]:
df['keywords'] = df.dominant_topic.apply(lambda x: topics_keywords[x])

In [ ]:
df.head(2)

,url,username,title,about,dominant_topic,topic_prob,keywords
0,https://www.linkedin.com/in/lokesh-kumar-xess-...,lokesh-kumar-xess-54814068,Marketing & Strategy at Mobistreak,I am a Google Adwords and Google Analytics cer...,5,0.184839,"[marketing, digital, brand, medium, content, o..."
1,https://in.linkedin.com/in/sujithnarayanan,sujithnarayanan,Reimagining Financial Services for India,Ideate. Execute. Disrupt. Iterate.,12,0.066239,"[business, sale, strategy, growth, market, str..."


# **Export Dataframe**

In [ ]:
df.to_csv('/content/drive/My Drive/NLP/with_swaraj/Data/lda_mallet_prediction/18_topics_pred_bigram_threshold_70.csv', index=False)

# **Visualization with pyLDAvis**

“pyLDAvis” is also a visualization library for presenting topic models. To use this library, you need to convert LdaMallet model to a gensim model. Below is the conversion method that I found on stackvverflow:


In [ ]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 4.2MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=f2fbee86419f46f999685f4144a30dc9fdfc531ee4eaf40dfcb4b88ce9fda772
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [ ]:
import pyLDAvis
import pyLDAvis.gensim

In [ ]:
# Converting lda_mallet to lda_model for pyLDAvis visualtization
from gensim.models.ldamodel import LdaModel

def convertldaMallet_to_ldagensim(mallet_model):
    model_gensim = LdaModel(
        id2word=mallet_model.id2word, num_topics=mallet_model.num_topics,
        alpha=mallet_model.alpha) 
    model_gensim.state.sstats[...] = mallet_model.wordtopics
    model_gensim.sync_state()
    return model_gensim

In [ ]:
ldagensim = convertldaMallet_to_ldagensim(ldamallet)

In [ ]:
import pyLDAvis.gensim as gensimvis
vis_data = gensimvis.prepare(ldagensim, corpus, id2word, sort_topics=False)
pyLDAvis.display(vis_data)

In [ ]:
ldamallet.print_topics()

[(0,
  '0.052*"work" + 0.041*"learn" + 0.041*"time" + 0.039*"good" + 0.033*"problem" + 0.028*"passionate" + 0.026*"challenge" + 0.025*"solve" + 0.023*"understand" + 0.020*"opportunity"'),
 (1,
  '0.027*"school" + 0.025*"social" + 0.024*"education" + 0.021*"world" + 0.018*"health" + 0.016*"student" + 0.016*"award" + 0.016*"university" + 0.015*"impact" + 0.015*"healthcare"'),
 (2,
  '0.050*"people" + 0.042*"make" + 0.037*"love" + 0.028*"life" + 0.023*"world" + 0.018*"thing" + 0.016*"find" + 0.016*"live" + 0.014*"great" + 0.014*"learn"'),
 (3,
  '0.191*"management" + 0.068*"customer" + 0.048*"manage" + 0.039*"process" + 0.034*"operation" + 0.031*"program" + 0.027*"client" + 0.025*"team" + 0.024*"support" + 0.020*"delivery"'),
 (4,
  '0.094*"team" + 0.047*"build" + 0.041*"lead" + 0.030*"leadership" + 0.028*"organization" + 0.028*"leader" + 0.027*"role" + 0.027*"people" + 0.025*"talent" + 0.024*"hr"'),
 (5,
  '0.169*"marketing" + 0.090*"digital" + 0.055*"brand" + 0.037*"medium" + 0.032*"con